In [1]:
import requests
import os
import pandas as pd
import json
import dotenv
import censusdata
import csv
dotenv.load_dotenv()

True

In [3]:
# get quandl api key
quandl_key = os.getenv('QUANDL_API_KEY')

In [4]:
# CSV county level population data downloaded from census.gov
population_df = pd.read_csv(open('co-est2019-alldata.csv'))
population_df

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2019,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,RNETMIG2016,RNETMIG2017,RNETMIG2018,RNETMIG2019
0,40,3,6,1,0,Alabama,Alabama,4779736,4780125,4785437,...,1.917501,0.578434,1.186314,1.522549,0.563489,0.626357,0.745172,1.090366,1.773786,2.483744
1,50,3,6,1,1,Alabama,Autauga County,54571,54597,54773,...,4.847310,6.018182,-6.226119,-3.902226,1.970443,-1.712875,4.777171,0.849656,0.540916,4.560062
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183112,...,24.017829,16.641870,17.488579,22.751474,20.184334,17.725964,21.279291,22.398256,24.727215,24.380567
3,50,3,6,1,5,Alabama,Barbour County,27457,27455,27327,...,-5.690302,0.292676,-6.897817,-8.132185,-5.140431,-15.724575,-18.238016,-24.998528,-8.754922,-5.165664
4,50,3,6,1,7,Alabama,Bibb County,22915,22915,22870,...,1.385134,-4.998356,-3.787545,-5.797999,1.331144,1.329817,-0.708717,-3.234669,-6.857092,1.831952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3188,50,4,8,56,37,Wyoming,Sweetwater County,43806,43806,43574,...,-17.605427,0.502513,15.035735,-4.614071,-13.140225,-13.762031,-18.484164,-23.515727,-18.743773,-17.065527
3189,50,4,8,56,39,Wyoming,Teton County,21294,21298,21296,...,-1.583464,-2.575509,1.905293,23.259519,12.508872,4.670450,1.253214,-0.171608,-12.432212,1.797445
3190,50,4,8,56,41,Wyoming,Uinta County,21118,21121,21089,...,-9.181105,-18.196975,-4.487730,-10.870861,-15.033634,-10.484550,-11.485101,-18.923455,-13.554993,-9.181105
3191,50,4,8,56,43,Wyoming,Washakie County,8533,8528,8530,...,-7.652085,-12.839390,-3.084589,-1.307811,-18.938032,0.000000,-15.204038,-15.950541,-17.246806,-8.289759


In [5]:
# census api
census_key = os.getenv('CENSUS_API_KEY')

# https://jtleider.github.io/censusdata/
censusdata.geographies(censusdata.censusgeo([('state', '36'), ('county', '081')]), 'acs5', 2010)
# censusdata.search('acs5', 2010, 'concept', 'Sex by Industry and Median Earnings for the Full-Time, Year-Round Civilian Employed Population 16 Years and Over')

# Sample B06004D.  Place of Birth (Asian Alone) in the United States
data = censusdata.download('acs5', 2015,
           censusdata.censusgeo([('state', '36'),# New York
                                 ('county', '081'),# Queens County
                                 ('block group', '*')]),
          ['B06004D_004E'])
data

,B06004D_004E
"Block Group 2, Census Tract 629, Queens County, New York: Summary level: 150, state:36> county:081> tract:062900> block group:2",None
"Block Group 3, Census Tract 629, Queens County, New York: Summary level: 150, state:36> county:081> tract:062900> block group:3",None
"Block Group 1, Census Tract 630, Queens County, New York: Summary level: 150, state:36> county:081> tract:063000> block group:1",None
"Block Group 2, Census Tract 630, Queens County, New York: Summary level: 150, state:36> county:081> tract:063000> block group:2",None
"Block Group 1, Census Tract 632, Queens County, New York: Summary level: 150, state:36> county:081> tract:063200> block group:1",None
...,...
"Block Group 1, Census Tract 596, Queens County, New York: Summary level: 150, state:36> county:081> tract:059600> block group:1",None
"Block Group 2, Census Tract 596, Queens County, New York: Summary level: 150, state:36> county:081> tract:059600> block group:2",None
"Block Group 1, Census Tract 598, Queens County, New York: Summary level: 150, state:36> county:081> tract:059800> block group:1",None
"Block Group 2, Census Tract 598, Queens County, New York: Summary level: 150, state:36> county:081> tract:059800> block group:2",None


In [6]:
# Reference Quandl Zillow Research Documentation
# https://www.quandl.com/data/ZILLOW-Zillow-Real-Estate-Research/documentation

# function for calling quandl zillow api
def fetch_zillow_quandl(quandl_code, quandl_key):
    url = f'https://www.quandl.com/api/v3/datasets/ZILLOW/{quandl_code}?start_date=2000-01-01&end_date=2019-12-31&api_key={quandl_key}'
    return json.loads(
        requests.get(url).content
    )['dataset']['data']

# State = S
# County = CO
# Greater Metropolitan Area = M
# City = C
# Neighborhood = N
# Zip Code = Z

def zillow_codes(path, delimiter):
    csv_data = csv.DictReader(open(path), delimiter=delimiter)
    return list(map(lambda x: dict(x), csv_data))

In [7]:
city_lookup = zillow_codes('Quandl_Zillow_Research/city.txt', '|')
county_lookup = zillow_codes('Quandl_Zillow_Research/county.txt', '|')
metro_lookup = zillow_codes('Quandl_Zillow_Research/metro.txt', '|')
state_lookup = zillow_codes('Quandl_Zillow_Research/state.txt', '|')
indicators_lookup = zillow_codes('Quandl_Zillow_Research/indicators.csv', '|')

In [8]:
fetch_zillow_quandl(
    'CO'+
    list(filter(lambda x: x['AREA'] == 'Nassau County, NY', county_lookup))[0]['CODE']+
    '_'+
    list(filter(lambda x: x['INDICATOR'] == 'Median Value Per Square Foot - All Homes', indicators_lookup))[0]['CODE'],
    quandl_key
)

[['2019-12-31', 632.0],
 ['2019-11-30', 631.0],
 ['2019-10-31', 631.0],
 ['2019-09-30', 632.0],
 ['2019-08-31', 632.0],
 ['2019-07-31', 631.0],
 ['2019-06-30', 631.0],
 ['2019-05-31', 632.0],
 ['2019-04-30', 632.0],
 ['2019-03-31', 631.0],
 ['2019-02-28', 630.0],
 ['2019-01-31', 629.0],
 ['2018-12-31', 628.0],
 ['2018-11-30', 626.0],
 ['2018-10-31', 623.0],
 ['2018-09-30', 619.0],
 ['2018-08-31', 616.0],
 ['2018-07-31', 613.0],
 ['2018-06-30', 611.0],
 ['2018-05-31', 608.0],
 ['2018-04-30', 604.0],
 ['2018-03-31', 600.0],
 ['2018-02-28', 596.0],
 ['2018-01-31', 593.0],
 ['2017-12-31', 591.0],
 ['2017-11-30', 589.0],
 ['2017-10-31', 587.0],
 ['2017-09-30', 585.0],
 ['2017-08-31', 582.0],
 ['2017-07-31', 578.0],
 ['2017-06-30', 572.0],
 ['2017-05-31', 567.0],
 ['2017-04-30', 564.0],
 ['2017-03-31', 562.0],
 ['2017-02-28', 560.0],
 ['2017-01-31', 557.0],
 ['2016-12-31', 553.0],
 ['2016-11-30', 551.0],
 ['2016-10-31', 548.0],
 ['2016-09-30', 546.0],
 ['2016-08-31', 543.0],
 ['2016-07-31', 